In [ ]:
import pandas as pd
import numpy as np

List = pd.read_excel('Sales.xlsx')

List

In [ ]:
#Determining Columns Name
Customer_ID = 'Customer_ID'
Order_Date = 'Purchase_Date'
Revenue = 'Net_Revenue'

In [ ]:
#Parameter Setting
Recency_weight=0.2
Frequency_weight=0.5
Monetary_weight=0.3

In [ ]:
#Recency
Recency = List.groupby(by='Customer_ID', as_index=False)[Order_Date].max()
Recency.columns = ['Customer_ID', 'Last_Order_Date']
Recency.Last_Order_Date = pd.to_datetime(Recency['Last_Order_Date'])
Last_date = Recency.Last_Order_Date.max()
Recency['Recency'] = Recency['Last_Order_Date'].apply(lambda x: (Last_date - x).days)
Recency

In [ ]:
#Frequency

Frequency = List.drop_duplicates().groupby(by=Customer_ID, as_index=False)[Order_Date ].count()
Frequency.columns = ['Customer_ID','Frequency']

In [ ]:
# Monetary

Monetary = List.groupby(by=Customer_ID, as_index=False)[Revenue].sum()
Monetary.columns = ['Customer_ID','Monetary']

In [ ]:
InitialMerged_List = Recency.merge(Frequency, on='Customer_ID')
result_RFM = InitialMerged_List.merge(Monetary, on='Customer_ID').drop(columns='Last_Order_Date')
result_RFM

In [ ]:
#Ranking

result_RFM['R_rank'] = result_RFM['Recency'].rank(ascending = False)
result_RFM['F_rank'] = result_RFM['Frequency'].rank(ascending = True)
result_RFM['M_rank'] = result_RFM['Monetary'].rank(ascending = True)
result_RFM

In [ ]:
#Scaling

result_RFM['R_rank_norm'] = (result_RFM['R_rank']/result_RFM['R_rank'].max())*100
result_RFM['F_rank_norm'] = (result_RFM['F_rank']/result_RFM['F_rank'].max())*100
result_RFM['M_rank_norm'] = (result_RFM['M_rank']/result_RFM['M_rank'].max())*100

result_RFM.drop(columns=['R_rank','F_rank','M_rank'], inplace=True)
result_RFM

In [ ]:
#Scoring

result_RFM['RFM_Score'] = Recency_weight*result_RFM['R_rank_norm']+Frequency_weight*result_RFM['F_rank_norm']+Monetary_weight*result_RFM['M_rank_norm']
result_RFM['RFM_Score'] *= 0.05
result_RFM = result_RFM.round(2)
result_RFM

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
X = result_RFM[['R_rank_norm','F_rank_norm','M_rank_norm']]

In [ ]:
inertia=[]
for i in range (2,15):
    kmn = KMeans(n_clusters=i, n_init=10 , random_state= 42)
    kmn.fit(X)
    inertia.append(kmn.inertia_)

# Plot
plt.figure(figsize=(8,6), dpi=150)
plt.plot(range(2,15),inertia, marker = 'o')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Curve')
plt.grid(True)
plt.show()

In [ ]:
# k = 8
best_kmeans = KMeans(n_clusters=8, n_init=10, random_state=42)
result_RFM['Cluster'] = best_kmeans.fit_predict(X)


In [ ]:
result_RFM.describe()

In [ ]:
Cluster_summary = result_RFM.groupby('Cluster').agg(
    {'Recency':'mean',
    'Frequency':'mean',
    'Monetary':'mean'}

).reset_index()

In [ ]:
print(Cluster_summary)

In [ ]:
result_RFM.describe()

In [ ]:
colors = ['#3498db', '#2ecc71', '#f39c12','#C9B1BD']

plt.figure(figsize=(8,8),dpi=150)

plt.subplot(3,1,1)
bars=plt.bar(Cluster_summary.index,Cluster_summary['Recency'], color= colors)
plt.xlabel('Cluster')
plt.ylabel('Avg Recency')
plt.title('Average Recency for Each Cluster')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(bars,Cluster_summary.index, title='Clusters')


plt.subplot(3,1,2)
bars = plt.bar(Cluster_summary.index,Cluster_summary['Frequency'], color= colors)
plt.xlabel('Cluster')
plt.ylabel('Avg Frequency')
plt.title('Average Frequency for Each Cluster')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(bars,Cluster_summary.index, title='Clusters')

plt.subplot(3,1,3)
bars = plt.bar(Cluster_summary.index,Cluster_summary['Monetary'], color= colors)
plt.xlabel('Cluster')
plt.ylabel('Avg Monetary')
plt.title('Average Monetary for Each Cluster')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(bars,Cluster_summary.index, title='Clusters')

plt.tight_layout()
plt.show()

In [ ]:
Labels = best_kmeans.labels_
Centroids = best_kmeans.cluster_centers_

plt.scatter(result_RFM['Recency'], result_RFM['Monetary'], s = 15, c = Labels)
#plt.scatter(Centroids[:, 0], Centroids[:, 2], s = 60, c = 'r', marker = 'x')
plt.xlabel('Recency')
plt.ylabel('Monetary')

In [ ]:
result_RFM.to_excel('RFM_Kmeans_Output.xlsx', index=False)